In [ ]:
import os
import json
import numpy as np

# ========= CONFIG =========
gt_dir = "dataset/labels/val"       # ground truth YOLO txt
pred_json = "runs/detect/val/predictions.json"  # generado por model.val(save_json=True)
iou_thresholds = np.arange(0.5, 1.0, 0.05)       # 0.50 ... 0.95
num_classes = 10  # <- AJUSTA
# ==========================

def yolo_to_xyxy(box, W, H):
    """Convierte YOLO (x,y,w,h) normalizado a coordenadas absolutas xyxy."""
    cx, cy, w, h = box
    x1 = (cx - w/2) * W
    y1 = (cy - h/2) * H
    x2 = (cx + w/2) * W
    y2 = (cy + h/2) * H
    return np.array([x1, y1, x2, y2])


def iou(box1, box2):
    """Calcula IoU entre dos cajas xyxy."""
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])

    inter = max(0, x2 - x1) * max(0, y2 - y1)
    area1 = (box1[2]-box1[0]) * (box1[3]-box1[1])
    area2 = (box2[2]-box2[0]) * (box2[3]-box2[1])
    union = area1 + area2 - inter

    return inter / union if union > 0 else 0


# ========= CARGAR GROUND TRUTH =========
gt = {}  # gt[image] = [(cls, box_xyxy), ...]

for txt in os.listdir(gt_dir):
    if not txt.endswith(".txt"):
        continue
    name = txt.replace(".txt", ".jpg")
    gt[name] = []
    with open(os.path.join(gt_dir, txt)) as f:
        for line in f:
            c, x, y, w, h = map(float, line.split())
            gt[name].append((int(c), np.array([x,y,w,h])))


# ========= CARGAR PREDICCIONES =========
with open(pred_json, "r") as f:
    preds_json = json.load(f)

preds = {}  # preds[image] = [(cls, conf, box_xyxy), ...]
for p in preds_json:
    name = p["image_path"].split("/")[-1]
    if name not in preds:
        preds[name] = []
    preds[name].append((p["class_id"], p["confidence"], np.array(p["bbox"])))


# ========= CALCULO DE AP =========

def compute_ap(rec, prec):
    """Area bajo curva PR (trapecios)."""
    return np.trapz(prec, rec)


def evaluate_class(c):
    """mAP para una clase concreta."""
    ap_per_iou = []

    for thr in iou_thresholds:
        TP = 0
        FP = 0
        FN = 0

        for image in gt.keys():
            gt_boxes = [b for cc,b in gt[image] if cc == c]
            pred_boxes = [p for cc,conf,p in preds.get(image, []) if cc == c]

            matched = [False]*len(gt_boxes)

            for pb in pred_boxes:
                found_match = False
                for i,gb in enumerate(gt_boxes):
                    if matched[i]:
                        continue
                    if iou(pb, gb) >= thr:
                        TP += 1
                        matched[i] = True
                        found_match = True
                        break
                if not found_match:
                    FP += 1

            FN += matched.count(False)

        # Evitar dividir por cero
        if TP+FP == 0:
            precision = 0
        else:
            precision = TP/(TP+FP)

        if TP+FN == 0:
            recall = 0
        else:
            recall = TP/(TP+FN)

        # curva PR con un solo punto → AP = P * R
        ap_per_iou.append(precision * recall)

    return np.mean(ap_per_iou)


# ========= mAP FINAL =========

AP = []
for c in range(num_classes):
    ap_c = evaluate_class(c)
    AP.append(ap_c)
    print(f"AP clase {c}: {ap_c:.4f}")

mAP = np.mean(AP)
